# This notebook will generate the Plausibility and Faithfulness results for the models

## The Expainability metrices are based on the work by DeYoung et. al. (2020) [ERASER: A Benchmark to Evaluate Rationalized NLP Models]

In [ ]:
# model choices include 'bert','bert_supervised','birnn','cnngru','birnn_att','birnn_scrat'
# attention lambda for bert_supervised is 0.001 and birnn_scrat is 100
# for testing_with_lime.py set number of samples is 100 (for faster inference set it lower but the results might not be consistent)

#python testing_with_rational.py <model> <att_lambda>
!python testing_with_rational.py bert_supervised 0.001
#python testing_with_lime.py <model> <number of samples> <att_lambda>
!python testing_with_lime.py bert_supervised 100 0.001

## <font color='red'>NOTE: Please generate the model explainability output before running this notebook</font>

In [1]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
import more_itertools as mit
import os

In [14]:
# get_annotated_data method is used to load the dataset
from Preprocess import *
from Preprocess.dataCollect import *

ModuleNotFoundError: No module named 'ekphrasis'

In [2]:
def get_annotated_data(params):
    #temp_read = pd.read_pickle(params['data_file'])
    with open(params['data_file'], 'r') as fp:
        data = json.load(fp)
    dict_data=[]
    for key in data:
        temp={}
        temp['post_id']=key
        temp['text']=data[key]['post_tokens']
        final_label=[]
        for i in range(1,4):
            temp['annotatorid'+str(i)]=data[key]['annotators'][i-1]['annotator_id']
#             temp['explain'+str(i)]=data[key]['annotators'][i-1]['rationales']
            temp['target'+str(i)]=data[key]['annotators'][i-1]['target']
            temp['label'+str(i)]=data[key]['annotators'][i-1]['label']
            final_label.append(temp['label'+str(i)])

        final_label_id=max(final_label,key=final_label.count)
        temp['rationales']=data[key]['rationales']
            
        if(params['class_names']=='Data/classes_two.npy'):
            if(final_label.count(final_label_id)==1):
                temp['final_label']='undecided'
            else:
                if(final_label_id in ['hatespeech','offensive']):
                    final_label_id='toxic'
                else:
                    final_label_id='non-toxic'
                temp['final_label']=final_label_id

        
        else:
            if(final_label.count(final_label_id)==1):
                temp['final_label']='undecided'
            else:
                temp['final_label']=final_label_id

        
        
        
        dict_data.append(temp)    
    temp_read = pd.DataFrame(dict_data)  
    return temp_read

In [3]:
dict_data_folder={
      '2':{'data_file':'data/dataset.json','class_label':'data/classes_two.npy'},
      '3':{'data_file':'data/dataset.json','class_label':'data/classes.npy'}
}

# We need to load the dataset with the labels as 'hatespeech', 'offensive', and 'normal' (3-class). 
params = {}
params['num_classes']=3
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [6]:
# The important key here is the 'bert_token'. Set it to True for Bert based models and False for Others.

params_data={
    'include_special':False,  #True is want to include <url> in place of urls if False will be removed
    'bert_tokens':False, #True /False
    'type_attention':'softmax', #softmax
    'set_decay':0.1,
    'majority':2,
    'max_length':128,
    'variance':10,
    'window':4,
    'alpha':0.5,
    'p_value':0.8,
    'method':'additive',
    'decay':False,
    'normalized':False,
    'not_recollect':True,
}


if(params_data['bert_tokens']):
    print('Loading BERT tokenizer...')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
else:
    print('Loading Normal tokenizer...')
    tokenizer=None

Loading Normal tokenizer...


In [9]:
def returnMask(row,params,tokenizer):
    
    text_tokens=row['text']
    
    
    
    ##### a very rare corner case
    if(len(text_tokens)==0):
        text_tokens=['dummy']
        print("length of text ==0")
    #####
    
    
    mask_all= row['rationales']
    mask_all_temp=mask_all
    count_temp=0
    while(len(mask_all_temp)!=3):
        mask_all_temp.append([0]*len(text_tokens))
    
    word_mask_all=[]
    word_tokens_all=[]
    
    for mask in mask_all_temp:
        if(mask[0]==-1):
            mask=[0]*len(mask)
        
        
        list_pos=[]
        mask_pos=[]
        
        flag=0
        for i in range(0,len(mask)):
            if(i==0 and mask[i]==0):
                list_pos.append(0)
                mask_pos.append(0)
            
            
            
            
            if(flag==0 and mask[i]==1):
                mask_pos.append(1)
                list_pos.append(i)
                flag=1
                
            elif(flag==1 and mask[i]==0):
                flag=0
                mask_pos.append(0)
                list_pos.append(i)
        if(list_pos[-1]!=len(mask)):
            list_pos.append(len(mask))
            mask_pos.append(0)
        string_parts=[]
        for i in range(len(list_pos)-1):
            string_parts.append(text_tokens[list_pos[i]:list_pos[i+1]])
        
        
        
        
        if(params['bert_tokens']):
            word_tokens=[101]
            word_mask=[0]
        else:
            word_tokens=[]
            word_mask=[]

        
        for i in range(0,len(string_parts)):
            tokens=ek_extra_preprocess(" ".join(string_parts[i]),params,tokenizer)
            masks=[mask_pos[i]]*len(tokens)
            word_tokens+=tokens
            word_mask+=masks


        if(params['bert_tokens']):
            ### always post truncation
            word_tokens=word_tokens[0:(int(params['max_length'])-2)]
            word_mask=word_mask[0:(int(params['max_length'])-2)]
            word_tokens.append(102)
            word_mask.append(0)

        word_mask_all.append(word_mask)
        word_tokens_all.append(word_tokens)
        
#     for k in range(0,len(mask_all)):
#          if(mask_all[k][0]==-1):
#             word_mask_all[k] = [-1]*len(word_mask_all[k])
    if(len(mask_all)==0):
        word_mask_all=[]
    else:    
        word_mask_all=word_mask_all[0:len(mask_all)]
    return word_tokens_all[0],word_mask_all    

In [12]:
def ek_extra_preprocess(text,params,tokenizer):
    remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
    word_list=text_processor.pre_process_doc(text)
    if(params['include_special']):
        pass
    else:
        word_list=list(filter(lambda a: a not in remove_words, word_list)) 
    if(params['bert_tokens']):
        sent=" ".join(word_list)
        sent = re.sub(r"[<\*>]", " ",sent)
        sub_word_list = custom_tokenize(sent,tokenizer)
        return sub_word_list
    else:            
        word_list=[token for token in word_list if token not in string.punctuation]
        return word_list


In [10]:
# Load the whole dataset and get the tokenwise rationales
def get_training_data(data):
    post_ids_list=[]
    text_list=[]
    attention_list=[]
    label_list=[]
    
    final_binny_output = []
    print('total_data',len(data))
    for index,row in tqdm(data.iterrows(),total=len(data)):
        annotation=row['final_label']
        
        text=row['text']
        post_id=row['post_id']
        annotation_list=[row['label1'],row['label2'],row['label3']]
        tokens_all = list(row['text'])
#         attention_masks =  [list(row['explain1']),list(row['explain2']),list(row['explain1'])]
        
        if(annotation!= 'undecided'):
            tokens_all,attention_masks=returnMask(row, params_data, tokenizer)
            final_binny_output.append([post_id, annotation, tokens_all, attention_masks, annotation_list])

    return final_binny_output



In [13]:
training_data=get_training_data(data_all_labelled)

total_data 20148


NameError: name 'text_processor' is not defined

In [7]:
training_data[19]

['18696652_gab',
 'offensive',
 [101,
  5310,
  25805,
  5582,
  4319,
  2224,
  2025,
  11382,
  3489,
  2012,
  2035,
  4283,
  2005,
  2008,
  19380,
  102],
 [[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
 ['hatespeech', 'offensive', 'offensive']]

In [3]:
# https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]
            
# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each==1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each)== int:
            start = each
            end = each+1
        elif len(each) == 2:
            start = each[0]
            end = each[1]+1
        else:
            print('error')

        output.append({"docid":post_id, 
              "end_sentence": -1, 
              "end_token": end, 
              "start_sentence": -1, 
              "start_token": start, 
              "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output

# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_split, save_path, id_division):  
    final_output = []
    
    if save_split:
        train_fp = open(save_path+'train.jsonl', 'w')
        val_fp = open(save_path+'val.jsonl', 'w')
        test_fp = open(save_path+'test.jsonl', 'w')
            
    for tcount, eachrow in enumerate(dataset):
        
        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]
        
        if majority_label=='normal':
            continue
        
        all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            explanations.append(list(each_explain))
        
        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]
        
            
        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)
        
        if save_split:
            if not os.path.exists(save_path+'docs'):
                os.makedirs(save_path+'docs')
            
            with open(save_path+'docs/'+post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))
            
            if post_id in id_division['train']:
                train_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['val']:
                val_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['test']:
                test_fp.write(json.dumps(temp)+'\n')
            else:
                print(post_id)
    
    if save_split:
        train_fp.close()
        val_fp.close()
        test_fp.close()
        
    return final_output

In [4]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
with open('./data/post_id_divisions.json') as fp:
    id_division = json.load(fp)

In [5]:
method = 'union'
save_split = True
save_path = './Data/Evaluation/Model_Eval/'  #The dataset in Eraser Format will be stored here.
convert_to_eraser_format(training_data, method, save_split, save_path, id_division)

NameError: name 'training_data' is not defined

### Generate the output

In [ ]:
%cd eraserbenchmark-master

In [ ]:
!ls

In [7]:
#--data_dir : Location of the folder which contains the dataset in eraser format
#--results : The location of the model output file in eraser format
#--score_file : The file name and location to write the output

!python rationale_benchmark/metrics.py --split test --strict --data_dir ../data --results ../data/output.jsonl --score_file ../data/model_explain_output.json

Traceback (most recent call last):
  File "rationale_benchmark/metrics.py", line 670, in <module>
    main()
  File "rationale_benchmark/metrics.py", line 610, in main
    docs = load_flattened_documents(args.data_dir, docids)
  File "C:\Users\kicop\OneDrive\Work\UvA_DataScience\QUVA\Development\G2X\hatexplain\eraserbenchmark\rationale_benchmark\utils.py", line 165, in load_flattened_documents
    unflattened_docs = load_documents(data_dir, docids)
  File "C:\Users\kicop\OneDrive\Work\UvA_DataScience\QUVA\Development\G2X\hatexplain\eraserbenchmark\rationale_benchmark\utils.py", line 152, in load_documents
    with open(os.path.join(docs_dir, d), 'r') as inf:
FileNotFoundError: [Errno 2] No such file or directory: '../data\\docs\\10068208_gab'


In [ ]:
# print the required results
with open('../model_explain_output.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])